# Multilayer Information Diffusion Simulator
---

This notebook implements a **Python-based simulator** for studying the impact of:

- Intra-layer clustering (via triangle formation),
- Multilayer (multiplex) structure, and
- Edge-based vs triangle-based diffusion

on **information diffusion dynamics in complex networks**.

It is designed to be consistent with the experimental setting described in the manuscript:

- Multilayer / multiplex complex network (e.g., Higgs Twitter, Citation network),
- Clustering controlled using an m-PageRank–based centrality,
- Diffusion through **single edges** and **triangular motifs**,
- Comparison of non-clustered (NN), semi-clustered (NC), and fully clustered (CC) configurations,
- Estimation of diffusion thresholds and final adoption size \(S(T)\) as a function of transmissibility \(T\).

> **Note:**  
> The real Higgs and Citation datasets are not bundled here.  
> Instead, this notebook provides a *generic, configurable simulator* that you can plug real data into (e.g., after constructing a NetworkX graph from your datasets).


In [ ]:
# 0. Basic setup and imports

import math
import random
from collections import defaultdict, deque
from typing import Dict, List, Tuple, Set

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

random.seed(42)
np.random.seed(42)

print("NetworkX version:", nx.__version__)


## 1. Helper utilities

These helpers provide:

- Construction of a *synthetic multiplex network* (two layers) mimicking degree and clustering levels.
- Computation of a **multilayer PageRank (m-PageRank)** centrality.
- Controlled creation / destruction of triangles to adjust the clustering coefficient while preserving (approximately) the degree distribution.


In [ ]:
# 1.1 Build a synthetic multiplex graph
#
# We use the same node set for both layers, but different edge sets.
# Each layer is a NetworkX graph stored in a dict: {layer_id: G_layer}.

def generate_multiplex_network(
    n: int = 2000,
    avg_degree_layer1: float = 4.0,
    avg_degree_layer2: float = 6.0,
) -> Dict[int, nx.Graph]:
    """Generate a simple 2-layer multiplex network using configuration-like graphs.

    Layer 1: relatively sparse
    Layer 2: denser (to mimic high-activity channels)
    """
    # Use a simple expected-degree model
    def _gnp(n, avg_degree):
        p = avg_degree / (n - 1)
        G = nx.fast_gnp_random_graph(n, p, seed=random.randint(0, 10**9))
        # retain the largest connected component
        if not nx.is_connected(G):
            giant = max(nx.connected_components(G), key=len)
            G = G.subgraph(giant).copy()
        return G

    G1 = _gnp(n, avg_degree_layer1)
    G2 = _gnp(n, avg_degree_layer2)
    print("Layer 1: nodes =", G1.number_of_nodes(), "edges =", G1.number_of_edges())
    print("Layer 2: nodes =", G2.number_of_nodes(), "edges =", G2.number_of_edges())
    return {0: G1, 1: G2}


# Example: generate a small multiplex for quick experimentation
layers = generate_multiplex_network(n=1000, avg_degree_layer1=4.0, avg_degree_layer2=6.0)


In [ ]:
# 1.2 Multilayer PageRank (m-PageRank)
#
# m-PageRank is defined here as an aggregation of PageRank scores
# computed separately on each layer. More sophisticated definitions
# (e.g., explicit inter-layer edges) can be added as needed.

def m_pagerank(
    layers: Dict[int, nx.Graph],
    alpha: float = 0.85,
    weight_by_layer: Dict[int, float] = None,
) -> Dict[int, float]:
    """Compute a simple multilayer PageRank by aggregating per-layer PageRank scores.

    Args:
        layers: dict mapping layer_id -> Graph (all with the same node set).
        alpha: PageRank damping factor.
        weight_by_layer: optional weights for each layer_id (default: uniform).

    Returns:
        dict: node -> m-PageRank score (normalized to sum=1).
    """
    if weight_by_layer is None:
        weight_by_layer = {lid: 1.0 for lid in layers}

    node_scores = defaultdict(float)
    for lid, G in layers.items():
        if G.number_of_nodes() == 0:
            continue
        pr = nx.pagerank(G, alpha=alpha)
        w = weight_by_layer.get(lid, 1.0)
        for v, score in pr.items():
            node_scores[v] += w * score

    # Normalize
    total = sum(node_scores.values())
    if total > 0:
        for v in list(node_scores.keys()):
            node_scores[v] /= total
    return dict(node_scores)


mpr = m_pagerank(layers)
print("Computed m-PageRank for", len(mpr), "nodes.")

# Show basic stats
vals = np.array(list(mpr.values()))
print("m-PageRank: min =", vals.min(), "max =", vals.max(), "mean =", vals.mean())


In [ ]:
# 1.3 Triangle utilities

def count_triangles(G: nx.Graph) -> int:
    """Return total number of triangles in an undirected graph."""
    tri_dict = nx.triangles(G)
    return sum(tri_dict.values()) // 3


def average_clustering(G: nx.Graph) -> float:
    return nx.average_clustering(G)


for lid, G in layers.items():
    print(f"Layer {lid}: C = {average_clustering(G):.4f}, triangles = {count_triangles(G)}")


In [ ]:
# 1.4 Increase or decrease clustering via triangle closing or edge rewiring

def increase_clustering(
    G: nx.Graph,
    mpr: Dict[int, float],
    budget_edges: int = 1000,
    close_probability: float = 0.5,
) -> nx.Graph:
    """Increase clustering by preferential triangle closing around high-mPR nodes.

    - For high-mPR nodes, we look at non-adjacent neighbor pairs and optionally
      add an edge to form a triangle.
    - The number of new edges is limited by budget_edges.
    """
    G = G.copy()
    nodes_sorted = sorted(G.nodes(), key=lambda v: -mpr.get(v, 0.0))
    added = 0

    for v in nodes_sorted:
        if added >= budget_edges:
            break
        nbrs = list(G.neighbors(v))
        if len(nbrs) < 2:
            continue
        # Try random pairs of neighbors
        random.shuffle(nbrs)
        for i in range(len(nbrs)):
            if added >= budget_edges:
                break
            for j in range(i + 1, len(nbrs)):
                u, w = nbrs[i], nbrs[j]
                if not G.has_edge(u, w) and random.random() < close_probability:
                    G.add_edge(u, w)
                    added += 1
                    if added >= budget_edges:
                        break
    return G


def decrease_clustering(
    G: nx.Graph,
    budget_swaps: int = 1000,
) -> nx.Graph:
    """Decrease clustering while approximately preserving degree distribution.

    Uses double-edge swaps to randomize the graph, which typically lowers clustering.
    """
    G = G.copy()
    if G.number_of_edges() < 2:
        return G
    # Perform edge swaps
    nx.double_edge_swap(G, nswap=budget_swaps, max_tries=5 * budget_swaps)
    return G


## 2. Construct NN, NC, and CC configurations

We now construct three variants of the multiplex network:

- **NN (Non-clustered)** — low clustering in both layers  
- **NC (Semi-clustered)** — one clustered layer, one non-clustered  
- **CC (Fully clustered)** — both layers highly clustered  

Clustering is controlled using:

- **m-PageRank** to prioritize influential nodes for triangle closing,
- **Edge swaps** to reduce clustering while preserving degrees.


In [ ]:
def build_NN_NC_CC(
    layers: Dict[int, nx.Graph],
    mpr: Dict[int, float],
    inc_budget: int = 2000,
    dec_budget: int = 4000,
):
    """Build NN, NC, CC configurations from base layers.

    Returns:
        configs: dict with keys 'NN', 'NC', 'CC', each a dict layer_id -> Graph
    """
    base_layers = {lid: G.copy() for lid, G in layers.items()}

    # First, create a low-clustered baseline for each layer
    low_layers = {lid: decrease_clustering(G, budget_swaps=dec_budget) for lid, G in base_layers.items()}

    # Then, create high-clustered versions using mPR-based triangle closing
    high_layers = {lid: increase_clustering(G, mpr, budget_edges=inc_budget) for lid, G in base_layers.items()}

    NN = {0: low_layers[0].copy(), 1: low_layers[1].copy()}
    NC = {0: high_layers[0].copy(), 1: low_layers[1].copy()}
    CC = {0: high_layers[0].copy(), 1: high_layers[1].copy()}

    return {"NN": NN, "NC": NC, "CC": CC}


configs = build_NN_NC_CC(layers, mpr, inc_budget=1500, dec_budget=3000)

for name, cfg in configs.items():
    print(f"Configuration {name}:")
    for lid, G in cfg.items():
        print(f"  Layer {lid}: C = {average_clustering(G):.4f}, triangles = {count_triangles(G)}")
    print()


## 3. Diffusion models

We now define:

- **Edge-based independent cascade diffusion** (single-edge contagion),
- **Triangle-based diffusion**, where activation is driven by triangular motifs.

For triangle-based diffusion, we adopt a simple but interpretable mechanism:

- Activation can occur via **either**:
  - a direct infected neighbor through an edge with probability \(T_\alpha\), or
  - a *reinforced* influence when two infected neighbors share a triangle with a susceptible node, modeled by a higher effective probability \(T_\beta\).

This captures the intuition of the analytical functions \(h_\alpha^t(x)\) and \(h_\beta^t(x)\) that separately account for single-edge and triangle contributions.


In [ ]:
# 3.1 Extract triangles and triangle adjacency structure

def enumerate_triangles(G: nx.Graph) -> List[Tuple[int, int, int]]:
    """Return list of triangles as node triples (u, v, w)."""
    # Use built-in algorithm
    triangles = []
    # networkx.cycle_basis can be used, but nx.triangles is faster
    # We'll build from node-wise neighbor lists
    nodes = list(G.nodes())
    node_index = {v: i for i, v in enumerate(nodes)}
    # Use a simple algorithm through adjacency
    for v in nodes:
        nbrs = list(G.neighbors(v))
        nbrs_sorted = sorted(nbrs, key=lambda x: node_index[x])
        for i in range(len(nbrs_sorted)):
            for j in range(i + 1, len(nbrs_sorted)):
                u, w = nbrs_sorted[i], nbrs_sorted[j]
                if G.has_edge(u, w):
                    tri = tuple(sorted((u, v, w), key=lambda x: node_index[x]))
                    triangles.append(tri)
    # Remove duplicates
    triangles = list(dict.fromkeys(triangles))
    return triangles


def build_triangle_index(triangles: List[Tuple[int, int, int]]):
    """Build mapping from node to triangles containing it."""
    tri_index = defaultdict(list)
    for idx, tri in enumerate(triangles):
        for v in tri:
            tri_index[v].append(idx)
    return tri_index


In [ ]:
# 3.2 Edge-based independent cascade diffusion

def simulate_edge_diffusion(
    G: nx.Graph,
    seeds: List[int],
    T: float,
    max_steps: int = 1000,
) -> Set[int]:
    """Independent cascade on edges with transmissibility T.

    Returns:
        infected: set of activated nodes at the end of the process.
    """
    infected = set(seeds)
    newly_infected = set(seeds)

    step = 0
    while newly_infected and step < max_steps:
        next_new = set()
        for u in newly_infected:
            for v in G.neighbors(u):
                if v not in infected:
                    if random.random() < T:
                        infected.add(v)
                        next_new.add(v)
        newly_infected = next_new
        step += 1
    return infected


In [ ]:
# 3.3 Triangle-based diffusion with reinforcement

def simulate_triangle_diffusion(
    G: nx.Graph,
    seeds: List[int],
    T_alpha: float,
    T_beta: float,
    max_steps: int = 1000,
) -> Set[int]:
    """Diffusion with both single-edge (T_alpha) and triangle-reinforced (T_beta) contagion.

    Mechanism:
    - For each susceptible node v and each infected neighbor u:
      - Try infection with probability T_alpha.
    - Additionally, if v is part of any triangle (v, u, w) where both u and w
      are infected, we attempt infection with probability T_beta.

    Returns:
        infected: set of activated nodes at the end of the process.
    """
    triangles = enumerate_triangles(G)
    tri_index = build_triangle_index(triangles)

    infected = set(seeds)
    newly_infected = set(seeds)

    step = 0
    while newly_infected and step < max_steps:
        next_new = set()

        # Edge-based trials
        for u in newly_infected:
            for v in G.neighbors(u):
                if v not in infected:
                    if random.random() < T_alpha:
                        infected.add(v)
                        next_new.add(v)

        # Triangle-based reinforcement
        # For every susceptible v, check triangles containing v
        for v in list(G.nodes()):
            if v in infected:
                continue
            in_tris = tri_index.get(v, [])
            if not in_tris:
                continue
            # If at least two nodes in a triangle are infected, try reinforced infection
            infected_neigh_pairs_found = False
            for tri_id in in_tris:
                tri = triangles[tri_id]
                others = [x for x in tri if x != v]
                if len(others) != 2:
                    continue
                u, w = others
                if u in infected and w in infected:
                    infected_neigh_pairs_found = True
                    if random.random() < T_beta:
                        infected.add(v)
                        next_new.add(v)
                        break
            if infected_neigh_pairs_found and v in infected:
                # Already infected in this step, no need to continue checking
                continue

        newly_infected = next_new
        step += 1

    return infected


## 4. Experimental pipeline

We now define an experiment driver to:

- Select a configuration (**NN**, **NC**, or **CC**),
- Run diffusion for a range of transmissibility values \(T\),
- Estimate the final adoption size \(S(T)\) and an empirical diffusion threshold.

### 4.1 Final adoption size and empirical threshold

For a given \(T\), we define:

- \( S(T) = \frac{|A(T)|}{N} \), where \(A(T)\) is the set of activated nodes.

A simple empirical **threshold** can be defined as the smallest \(T\) where:

- \( S(T) \ge S_{\min} \), e.g. \(S_{\min} = 0.1\) (10% adoption),
- You can adjust this according to your theoretical threshold from the PGF analysis.


In [ ]:
def run_diffusion_curve(
    G: nx.Graph,
    T_values: List[float],
    mode: str = "edge",
    T_alpha: float = None,
    T_beta: float = None,
    num_seeds: int = 5,
    num_reps: int = 5,
):
    """Run diffusion for each T and return S(T) for chosen mode.

    mode: 'edge' or 'triangle'
    """
    n = G.number_of_nodes()
    nodes = list(G.nodes())

    results = []
    for T in T_values:
        sizes = []
        for _ in range(num_reps):
            seeds = random.sample(nodes, num_seeds)
            if mode == "edge":
                infected = simulate_edge_diffusion(G, seeds, T=T)
            elif mode == "triangle":
                Ta = T if T_alpha is None else T_alpha
                Tb = min(1.0, T * 1.5) if T_beta is None else T_beta
                infected = simulate_triangle_diffusion(G, seeds, T_alpha=Ta, T_beta=Tb)
            else:
                raise ValueError("Unknown mode: " + mode)
            sizes.append(len(infected) / n)
        results.append(np.mean(sizes))
    return np.array(results)


def estimate_threshold(T_values: List[float], S_values: np.ndarray, S_min: float = 0.1):
    """Estimate empirical threshold as smallest T with S(T) >= S_min."""
    for T, S in zip(T_values, S_values):
        if S >= S_min:
            return T
    return None


### 4.2 Example: diffusion vs clustering on NN / NC / CC

We now:

- Select the **largest connected component** of each layer in a given configuration,
- Use one of the layers as the diffusion substrate (or you can merge them),
- Compare diffusion curves for:
  - Non-clustered (NN),
  - Semi-clustered (NC),
  - Fully clustered (CC).


In [ ]:
def get_largest_cc(G: nx.Graph) -> nx.Graph:
    if nx.is_connected(G):
        return G.copy()
    giant = max(nx.connected_components(G), key=len)
    return G.subgraph(giant).copy()


# Choose which layer to simulate diffusion on (e.g., layer 0)
layer_id_for_diffusion = 0

T_values = np.linspace(0.0, 1.0, 21)

curves_edge = {}
curves_tri = {}
thresholds_edge = {}
thresholds_tri = {}

for name, cfg in configs.items():
    G_layer = get_largest_cc(cfg[layer_id_for_diffusion])
    print(f"{name}: using layer {layer_id_for_diffusion} with n={G_layer.number_of_nodes()}, "
          f"m={G_layer.number_of_edges()}, C={average_clustering(G_layer):.4f}")

    # Edge-based diffusion
    S_edge = run_diffusion_curve(
        G_layer, T_values, mode="edge", num_seeds=5, num_reps=5
    )
    curves_edge[name] = S_edge
    thresholds_edge[name] = estimate_threshold(T_values, S_edge, S_min=0.1)

    # Triangle-based diffusion
    S_tri = run_diffusion_curve(
        G_layer, T_values, mode="triangle", num_seeds=5, num_reps=5
    )
    curves_tri[name] = S_tri
    thresholds_tri[name] = estimate_threshold(T_values, S_tri, S_min=0.1)


### 4.3 Plotting diffusion curves

The following cells visualize:

- Final adoption size \(S(T)\) vs transmissibility \(T\) for different clustering configurations,
- Empirical thresholds estimated from the simulation.


In [ ]:
# Edge-based curves

plt.figure()
for name, S in curves_edge.items():
    plt.plot(T_values, S, label=f"{name}")
plt.xlabel("Transmissibility T")
plt.ylabel("Final adoption size S(T)")
plt.title("Edge-based diffusion: S(T) vs T for NN / NC / CC")
plt.legend()
plt.grid(True)
plt.show()

print("Empirical thresholds (edge-based, S_min=0.1):")
for name, th in thresholds_edge.items():
    print(f"  {name}: T_c ≈ {th}")


In [ ]:
# Triangle-based curves

plt.figure()
for name, S in curves_tri.items():
    plt.plot(T_values, S, label=f"{name}")
plt.xlabel("Transmissibility T")
plt.ylabel("Final adoption size S(T)")
plt.title("Triangle-based diffusion: S(T) vs T for NN / NC / CC")
plt.legend()
plt.grid(True)
plt.show()

print("Empirical thresholds (triangle-based, S_min=0.1):")
for name, th in thresholds_tri.items():
    print(f"  {name}: T_c ≈ {th}")


## 5. Higgs-like subset experiment (10,000 nodes)

This section reproduces the *spirit* of the theoretical validation experiment:

- Extract a largest weakly connected component (WCC) with approximately:
  - \(N \approx 10^4\) nodes,
  - \(E \approx 3.4 \times 10^4\) edges.
- Simulate diffusion for:
  - **single-edge mode (h\_\alpha-like)**,
  - **triangle-enhanced mode (h\_\beta-like)**.

In practice, you can replace this synthetic graph with the actual Higgs Twitter WCC.


In [ ]:
# 5.1 Generate a Higgs-like synthetic network

def generate_higgs_like_graph(
    N: int = 10_000,
    avg_degree: float = 6.8,  # so E ≈ N * avg_degree / 2 ≈ 3.4e4
) -> nx.Graph:
    p = avg_degree / (N - 1)
    G = nx.fast_gnp_random_graph(N, p, seed=123)
    # Take largest connected component
    if not nx.is_connected(G):
        giant = max(nx.connected_components(G), key=len)
        G = G.subgraph(giant).copy()
    return G


G_higgs_like = generate_higgs_like_graph()
print("Higgs-like graph:",
      "N =", G_higgs_like.number_of_nodes(),
      "E =", G_higgs_like.number_of_edges(),
      "C =", average_clustering(G_higgs_like),
      "triangles =", count_triangles(G_higgs_like))


In [ ]:
# 5.2 Run S(T) curves for edge-only vs triangle-enhanced diffusion

T_values_higgs = np.linspace(0.0, 1.0, 21)

S_edge_higgs = run_diffusion_curve(
    G_higgs_like,
    T_values_higgs,
    mode="edge",
    num_seeds=10,
    num_reps=3,
)

S_tri_higgs = run_diffusion_curve(
    G_higgs_like,
    T_values_higgs,
    mode="triangle",
    num_seeds=10,
    num_reps=3,
)

T_edge_th = estimate_threshold(T_values_higgs, S_edge_higgs, S_min=0.1)
T_tri_th = estimate_threshold(T_values_higgs, S_tri_higgs, S_min=0.1)

print("Empirical thresholds on Higgs-like graph (S_min=0.1):")
print("  Edge-based   T_c ≈", T_edge_th)
print("  Triangle-enh T_c ≈", T_tri_th)


In [ ]:
# 5.3 Plot S(T) curves for Higgs-like graph

plt.figure()
plt.plot(T_values_higgs, S_edge_higgs, label="Edge-based (h_alpha-like)")
plt.plot(T_values_higgs, S_tri_higgs, label="Triangle-based (h_beta-like)")
plt.xlabel("Transmissibility T")
plt.ylabel("Final adoption size S(T)")
plt.title("Higgs-like WCC: Edge vs Triangle diffusion")
plt.legend()
plt.grid(True)
plt.show()


## 6. Connecting to analytical formulations

In the article, diffusion through single edges and triangles is described analytically using:

- Probability generating functions (PGFs),
- Functions \(h_\alpha^t(x)\) and \(h_\beta^t(x)\),
- Jacobian-based stability analysis to derive the epidemic threshold.

This notebook focuses on the **simulation** side. To integrate your exact analytical model:

1. Implement functions for \(h_\alpha^t(x)\), \(h_\beta^t(x)\), and the PGFs of the degree and triangle distributions.
2. Compute the Jacobian at the disease-free equilibrium and obtain the theoretical threshold \(T_c^{(theory)}\).
3. Compare it to the empirical thresholds estimated here (`estimate_threshold`).

You can add a new code cell below where you plug in your closed-form expressions
derived in the manuscript.
